In [1]:
import tools as tl
import pandas as pd
import numpy as np
import os
from scipy import spatial
from scipy.stats import pearsonr, spearmanr

In [2]:
os.chdir('ml-100k/')

Para esse conjunto de exercícios vamos utilizar o dataset MovieLens 100k

Os conjuntos de dados do MovieLens foram coletados pelo GroupLens Research Project na Universidade de Minnesota.

Este conjunto de dados consiste em:

* 100.000 classificações (1-5) de 943 usuários em 1.682 filmes.
* Cada usuário classificou pelo menos 20 filmes.
* Informações demográficas simples para os usuários (idade, sexo, ocupação, CEP)

Os dados foram coletados por meio do site MovieLens (movielens.umn.edu) durante o período de sete meses a partir de 19 de setembro, 1997 a 22 de abril de 1998. Estes dados foram limpos - usuários que tiveram menos de 20 avaliações ou não tiveram dados demográficos completos informações foram removidas deste conjunto de dados. Descrições detalhadas de o arquivo de dados pode ser encontrado no final deste arquivo.

Crie o seguinte conjunto de classes:

1. Classe Usuário contendo os dados relativos ao usuário (arquivo u.user) e com as seguintes responsabilidades:

* Cálculo do valor médio de avaliações;

In [3]:
class Usuario:
    def __init__(self, dados_users, columns_users, separator_users, dados_rat, columns_rat, separator_rat):
        self.users_pd = pd.read_csv(dados_users, names=columns_users, sep=separator_users)
        self.data_pd = pd.read_csv(dados_rat, names=columns_rat, sep=separator_rat)
        self.user_id = self.users_pd['user_id']
        self.age = self.users_pd['age']
        self.gender = self.users_pd['gender']
        self.occ = self.users_pd['occupation']
        self.zip_code = self.users_pd['zip_code']
    
    def dados_user(self, user):
        return self.users_pd.iloc[(user-1)]
    
    def occ_user(self, user):
        return self.users_pd['occupation'].iloc[(user-1)]
    
    def rating_user(self, user):
        items = list(self.data_pd[self.data_pd['user_id']==user]['item_id'])
        rats = list(self.data_pd[self.data_pd['user_id']==user]['rating'])
        rating_dict = dict(zip(items, rats))
        return rating_dict
        
          
    def rating_mean_user(self, user):
        mean_user = self.data_pd[self.data_pd['user_id']==user]['rating'].mean()
        return mean_user

In [4]:
users = Usuario(dados_users='u.user', columns_users=['user_id', 'age', 'gender', 'occupation', 'zip_code'], separator_users='|', 
                dados_rat='u.data', columns_rat=['user_id', 'item_id', 'rating', 'timestampsep'], separator_rat='\t')

In [5]:
users.users_pd

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [6]:
users.data_pd

,user_id,item_id,rating,timestampsep
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [7]:
users.rating_mean_user(191) # calcula a média de avaliações de um usuário informado

3.6296296296296298

In [8]:
users.users_pd.iloc[0]

user_id                1
age                   24
gender                 M
occupation    technician
zip_code           85711
Name: 0, dtype: object

In [9]:
users.users_pd['age']

0      24
1      53
2      23
3      24
4      33
       ..
938    26
939    32
940    20
941    48
942    22
Name: age, Length: 943, dtype: int64

In [10]:
users.age

0      24
1      53
2      23
3      24
4      33
       ..
938    26
939    32
940    20
941    48
942    22
Name: age, Length: 943, dtype: int64

In [11]:
users.dados_user(7)

user_id                   7
age                      57
gender                    M
occupation    administrator
zip_code              91344
Name: 6, dtype: object

2. Classe Filme contendo os dados relativos ao filme (arquivo u.item) e com as seguintes responsabilidades:

* Cálculo da avaliação média;

In [12]:
class Filme:
    
    def __init__(self,  dados_film, columns_film, separator_film, dados_rat, columns_rat, separator_rat):
        self.films_pd = pd.read_csv(dados_film, names=columns_film, sep=separator_film)
        self.data_pd = pd.read_csv(dados_rat, names=columns_rat, sep=separator_rat)
    
    def dados_film(self, item):
        return self.films_pd.iloc[(item-1)]
    
    def rating_mean_film(self, item):
        mean_user = self.data_pd[self.data_pd['item_id']==item]['rating'].mean()
        return mean_user
    
    def genre_film(self, item):
        genre = list(self.films_pd.columns)[5:]
        genre_item = []

        for i in genre:
            if self.films_pd[i].iloc[item-1] == 1:
                genre_item.append(i)
        
        return genre_item
    

In [13]:
filmes = Filme(dados_film='u.item', columns_film=['movie_id', 'movie_title', 'release_date', 'video_release_date','IMDb_URL', 'unknown', 'Action', 
                                  'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film_Noir', 
                                  'Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western'], separator_film='|',
               dados_rat='u.data', columns_rat=['user_id', 'item_id', 'rating', 'timestampsep'], separator_rat='\t')

In [14]:
filmes.dados_film(2)

movie_id                                                              2
movie_title                                            GoldenEye (1995)
release_date                                                01-Jan-1995
video_release_date                                                  NaN
IMDb_URL              http://us.imdb.com/M/title-exact?GoldenEye%20(...
unknown                                                               0
Action                                                                1
Adventure                                                             1
Animation                                                             0
Childrens                                                             0
Comedy                                                                0
Crime                                                                 0
Documentary                                                           0
Drama                                                           

In [15]:
filmes.rating_mean_film(1)  # média de avaliação de um dado usuário

3.8783185840707963

3. Classe Gênero contendo os dados relacionados ao gênero do filme (arquivo u.genre)

Não se adicionou uma nova classe genero, criou-se um método que retorna o genero de um dado item_id (`genre_film`)

In [16]:
filmes.genre_film(1)

['Animation', 'Childrens', 'Comedy']

4. Classe Profissão contendo dados relacionados a ocupação dos usuários;


Não se adicionou uma nova classe profissões, criou-se um método que retorna o genero de um dado user_id (`occ_user`)

In [17]:
users.occ_user(1)

'technician'

5. Classe Avaliação contendo dados das avaliações dos usuários (arquivo u.data);

In [18]:
users.rating_user(196) # retorna um dicionário com o item_id e a nota do usuário informado

{242: 3,
 393: 4,
 381: 4,
 251: 3,
 655: 5,
 67: 5,
 306: 4,
 238: 4,
 663: 5,
 111: 4,
 580: 2,
 25: 4,
 286: 5,
 94: 3,
 692: 5,
 8: 5,
 428: 4,
 1118: 4,
 70: 3,
 66: 3,
 257: 2,
 108: 4,
 202: 3,
 340: 3,
 287: 3,
 116: 3,
 382: 4,
 285: 5,
 1241: 3,
 1007: 4,
 411: 4,
 153: 5,
 13: 2,
 762: 3,
 173: 2,
 1022: 4,
 845: 4,
 269: 3,
 110: 1}

6. Classe SistemaDeRecomendação com as seguintes responsabilidades:

* Manter lista de usuários;
* Manter lista de filmes;
* Manter Lista de profissões;
* Manter Lista de gêneros;
* Adicionar novo usuário;
* Adicionar novo filme;
* Adicionar nova avaliação de filme;
